# Data Process

In [13]:
import sys
sys.path.append('../')
import os
import csv
from dataprocess import diag_pro_codes
from dataprocess.discharge_summary import get_disch
from dataprocess import concat_code_disch
import word_embedding
from dataprocess import load_code_description
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim import models
from collections import Counter
import operator

In [9]:
#path of data
data_dir = os.path.join(os.getcwd(),'mimicdata/mimic3')
dev_data_dir = os.path.join(os.getcwd(),'dev_data') #dir for generated data
pro_dir = os.path.join(data_dir,'PROCEDURES_ICD.csv') 
diag_dir = os.path.join(data_dir,'DIAGNOSES_ICD.csv') 
notes_dir = os.path.join(data_dir,'NOTEEVENTS.csv') 
all_code_dir = os.path.join(dev_data_dir,'ALL_ICD.csv')
disch_sum_dir = os.path.join(dev_data_dir,'DISCH_SUM.csv') 
code_filtered_dir = os.path.join(dev_data_dir,'ALL_CODES_FILTERED.csv')
all_code_disch_dir = os.path.join(dev_data_dir,'ALL_CODES_NOTES.csv')
train_dir = os.path.join(dev_data_dir,'train_dataset.csv')
model_dir = os.path.join(os.getcwd(),'models/embeding')
vocab_tr_dir = os.path.join(dev_data_dir,'VOCAB_TRAIN.csv')
vocab_matrix_dir = os.path.join(dev_data_dir,'vocab_matrix.csv')

In [4]:
#1. create ALL_CODES.csv: contains both procedure and diagnoses ICD_CODES.
#cannot be deleted!!!
#diag_pro_codes.diag_pro_codes(pro_dir,diag_dir)
dficd_codes = pd.read_csv(all_code_dir,dtype={"ICD9_formatted": str})
#print(len(dficd_codes['ICD9_formatted'].unique()))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
#2. create discharge summary data: DISCH_SUM.csv
#cannot be deleted!!!
#get_disch(notes_dir)
df_discharge = pd.read_csv(disch_sum_dir)

In [47]:
print(len(df_discharge['HADM_ID'].unique()),len(dficd_codes['HADM_ID'].unique()))
#3. filter out data whose 'HADM_ID' not in df_discharge
#cannot be deleted!!!
#diag_pro_codes.code_filter(df,df_discharge)
dficd_codes_filtered = pd.read_csv(code_filtered_dir,index_col=None,dtype={"ICD9_CODE": str})
print(len(dficd_codes_filtered['HADM_ID'].unique()))
#print(len(dficd_codes_filtered['ICD9_CODE'].unique()))

52726 58976
52726
8885


In [6]:
#4. Combine discharge and icd_codes
all_code_disch_dir = concat_code_disch.get_code_disch(dficd_codes_filtered,df_discharge)
dfall_code_disch = pd.read_csv(all_code_disch_dir)


  0%|          | 9/52726 [00:00<10:14, 85.83it/s]

(52726, 2) & (52726, 2)


100%|██████████| 52726/52726 [09:21<00:00, 93.91it/s] 


In [7]:
#5. split the dataset
train,test,dev = concat_code_disch.split_dataset(dfall_code_disch)


  0%|          | 56/47723 [00:00<01:25, 554.44it/s]

Processing train dataset ... 


  2%|▏         | 73/3372 [00:00<00:04, 719.00it/s]

Processing test dataset ... 


  5%|▍         | 75/1631 [00:00<00:02, 738.02it/s]

Processing dev dataset ... 


100%|██████████| 1631/1631 [00:02<00:00, 776.05it/s]


In [8]:
#get training, testing, validation sets:
split_dir = os.path.join(os.getcwd(),'dev_data')
sets = ['train','test','dev'] 
dataset = [os.path.join(split_dir,'%s_dataset.csv'%st) for st in sets]
train,test,dev = dataset
train_set = pd.read_csv(train)
test_set = pd.read_csv(test)
dev_set = pd.read_csv(dev)
print('Training set: '+str(train_set.shape[0])+'\n'
     +'Testing set: '+str(test_set.shape[0])+'\n'
      +'Validation set: '+str(dev_set.shape[0]))
train_set.shape[0]+test_set.shape[0]+dev_set.shape[0]

Training set: 47723
Testing set: 3372
Validation set: 1631


52726

In [9]:
train_set

,SUBJECT_ID,HADM_ID,TEXT,LABELS
0,158,169433,admission date discharge date date of birth s...,44.43;532.40;493.20;V45.81;412;401.9
1,2896,178124,name known lastname known firstname unit no n...,45.13;45.25;211.3;427.31;578.9;560.1;496;584.9...
2,6495,139808,admission date discharge date date of birth s...,86.22;86.69;83.82;83.43;93.59;998.59;998.32;90...
3,3564,117638,admission date discharge date service doctor ...,55.92;96.72;11.;31.42;038.49;041.6;785.59;518....
4,7995,190945,admission date discharge date date of birth s...,39.29;88.48;440.22;492.8;401.9;714.0
5,70734,124010,name known lastname known firstname unit no n...,79.04;88.47;852.20;410.71;805.6;285.1;816.12;E...
6,11808,129572,name known lastname known firstname unit no n...,96.71;96.04;428.0;518.81;496;401.0;250.40;583.81
7,7389,145223,admission date discharge date date of birth s...,96.04;96.71;464.51;292.0;202.80
8,18860,114195,admission date discharge date date of birth s...,36.12;35.21;36.15;39.61;414.01;424.1;401.9;272.0
9,7908,182396,admission date discharge date date of birth s...,42.33;45.13;572.2;038.49;785.52;584.9;585;571....


# Word_Embedding

In [22]:
#1. get vocabulary in training set
word_embedding.build_vocab(train_dir) #create VOCAB_TRAIN.csv'
vocab_tr = pd.read_csv(vocab_tr_dir)    


In [24]:
#vocab_tr

In [3]:
#2. word_embedding: get vector for each word
#train the model to get word_vectors
#word_embedding.build_model(disch_sum_dir)


2018-06-30 13:01:29,360 : INFO : collecting all words and their counts
2018-06-30 13:01:29,366 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-30 13:01:36,615 : INFO : PROGRESS: at sentence #10000, processed 14394563 words, keeping 61963 word types
2018-06-30 13:01:42,783 : INFO : PROGRESS: at sentence #20000, processed 26406371 words, keeping 82263 word types
2018-06-30 13:01:49,216 : INFO : PROGRESS: at sentence #30000, processed 39523646 words, keeping 102399 word types
2018-06-30 13:01:56,555 : INFO : PROGRESS: at sentence #40000, processed 54242571 words, keeping 122662 word types
2018-06-30 13:02:04,547 : INFO : PROGRESS: at sentence #50000, processed 70115224 words, keeping 141261 word types
2018-06-30 13:02:09,345 : INFO : collected 150854 word types from a corpus of 79801387 raw words and 59652 sentences
2018-06-30 13:02:09,346 : INFO : Loading a fresh vocabulary
2018-06-30 13:02:09,958 : INFO : min_count=0 retains 150854 unique words (100% o

2018-06-30 13:03:12,995 : INFO : EPOCH 1 - PROGRESS: at 66.89% examples, 755383 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:03:14,003 : INFO : EPOCH 1 - PROGRESS: at 67.95% examples, 756382 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:03:15,006 : INFO : EPOCH 1 - PROGRESS: at 68.92% examples, 757022 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:03:16,019 : INFO : EPOCH 1 - PROGRESS: at 69.98% examples, 757895 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:03:17,029 : INFO : EPOCH 1 - PROGRESS: at 70.97% examples, 758483 words/s, in_qsize 5, out_qsize 2
2018-06-30 13:03:18,042 : INFO : EPOCH 1 - PROGRESS: at 72.02% examples, 759759 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:03:19,052 : INFO : EPOCH 1 - PROGRESS: at 73.00% examples, 759650 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:03:20,058 : INFO : EPOCH 1 - PROGRESS: at 74.00% examples, 759723 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:03:21,066 : INFO : EPOCH 1 - PROGRESS: at 74.96% examples, 759695 words/s, in_qsiz

2018-06-30 13:04:22,791 : INFO : EPOCH 2 - PROGRESS: at 51.14% examples, 789485 words/s, in_qsize 6, out_qsize 0
2018-06-30 13:04:23,801 : INFO : EPOCH 2 - PROGRESS: at 52.18% examples, 790228 words/s, in_qsize 7, out_qsize 1
2018-06-30 13:04:24,815 : INFO : EPOCH 2 - PROGRESS: at 53.19% examples, 790387 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:04:25,827 : INFO : EPOCH 2 - PROGRESS: at 54.13% examples, 788947 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:04:26,839 : INFO : EPOCH 2 - PROGRESS: at 55.30% examples, 788479 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:04:27,844 : INFO : EPOCH 2 - PROGRESS: at 56.45% examples, 788629 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:04:28,859 : INFO : EPOCH 2 - PROGRESS: at 57.64% examples, 788916 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:04:29,876 : INFO : EPOCH 2 - PROGRESS: at 58.62% examples, 785835 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:04:30,893 : INFO : EPOCH 2 - PROGRESS: at 59.87% examples, 786370 words/s, in_qsiz

2018-06-30 13:05:32,616 : INFO : EPOCH 3 - PROGRESS: at 34.29% examples, 786700 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:05:33,634 : INFO : EPOCH 3 - PROGRESS: at 35.70% examples, 786525 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:05:34,635 : INFO : EPOCH 3 - PROGRESS: at 36.93% examples, 786645 words/s, in_qsize 8, out_qsize 0
2018-06-30 13:05:35,637 : INFO : EPOCH 3 - PROGRESS: at 38.09% examples, 786262 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:05:36,640 : INFO : EPOCH 3 - PROGRESS: at 39.30% examples, 786503 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:05:37,654 : INFO : EPOCH 3 - PROGRESS: at 40.54% examples, 787172 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:05:38,659 : INFO : EPOCH 3 - PROGRESS: at 41.81% examples, 787836 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:05:39,672 : INFO : EPOCH 3 - PROGRESS: at 43.26% examples, 788666 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:05:40,690 : INFO : EPOCH 3 - PROGRESS: at 44.70% examples, 788723 words/s, in_qsiz

2018-06-30 13:06:42,416 : INFO : EPOCH 4 - PROGRESS: at 15.67% examples, 792305 words/s, in_qsize 7, out_qsize 1
2018-06-30 13:06:43,442 : INFO : EPOCH 4 - PROGRESS: at 16.84% examples, 793083 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:06:44,442 : INFO : EPOCH 4 - PROGRESS: at 18.00% examples, 794011 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:06:45,449 : INFO : EPOCH 4 - PROGRESS: at 19.14% examples, 794606 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:06:46,454 : INFO : EPOCH 4 - PROGRESS: at 20.54% examples, 795231 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:06:47,462 : INFO : EPOCH 4 - PROGRESS: at 22.01% examples, 796240 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:06:48,464 : INFO : EPOCH 4 - PROGRESS: at 23.52% examples, 797721 words/s, in_qsize 6, out_qsize 0
2018-06-30 13:06:49,470 : INFO : EPOCH 4 - PROGRESS: at 24.86% examples, 797205 words/s, in_qsize 6, out_qsize 0
2018-06-30 13:06:50,472 : INFO : EPOCH 4 - PROGRESS: at 26.18% examples, 794949 words/s, in_qsiz

2018-06-30 13:07:52,061 : INFO : EPOCH - 4 : training on 79801387 raw words (66429816 effective words) took 83.8s, 793125 effective words/s
2018-06-30 13:07:53,072 : INFO : EPOCH 5 - PROGRESS: at 1.06% examples, 778477 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:07:54,073 : INFO : EPOCH 5 - PROGRESS: at 2.19% examples, 801351 words/s, in_qsize 5, out_qsize 0
2018-06-30 13:07:55,087 : INFO : EPOCH 5 - PROGRESS: at 3.22% examples, 785350 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:07:56,096 : INFO : EPOCH 5 - PROGRESS: at 4.29% examples, 781100 words/s, in_qsize 7, out_qsize 1
2018-06-30 13:07:57,097 : INFO : EPOCH 5 - PROGRESS: at 5.37% examples, 783279 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:07:58,100 : INFO : EPOCH 5 - PROGRESS: at 6.44% examples, 789066 words/s, in_qsize 4, out_qsize 0
2018-06-30 13:07:59,105 : INFO : EPOCH 5 - PROGRESS: at 7.52% examples, 791961 words/s, in_qsize 5, out_qsize 0
2018-06-30 13:08:00,106 : INFO : EPOCH 5 - PROGRESS: at 8.52% examples, 7844

2018-06-30 13:09:05,637 : INFO : EPOCH 5 - PROGRESS: at 83.09% examples, 785064 words/s, in_qsize 0, out_qsize 0
2018-06-30 13:09:06,639 : INFO : EPOCH 5 - PROGRESS: at 84.04% examples, 784915 words/s, in_qsize 3, out_qsize 0
2018-06-30 13:09:07,641 : INFO : EPOCH 5 - PROGRESS: at 84.99% examples, 784807 words/s, in_qsize 8, out_qsize 0
2018-06-30 13:09:08,644 : INFO : EPOCH 5 - PROGRESS: at 85.97% examples, 784690 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:09:09,653 : INFO : EPOCH 5 - PROGRESS: at 87.05% examples, 784735 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:09:10,664 : INFO : EPOCH 5 - PROGRESS: at 88.05% examples, 784586 words/s, in_qsize 7, out_qsize 0
2018-06-30 13:09:11,674 : INFO : EPOCH 5 - PROGRESS: at 89.10% examples, 784499 words/s, in_qsize 5, out_qsize 2
2018-06-30 13:09:12,674 : INFO : EPOCH 5 - PROGRESS: at 90.16% examples, 784557 words/s, in_qsize 6, out_qsize 1
2018-06-30 13:09:13,677 : INFO : EPOCH 5 - PROGRESS: at 91.24% examples, 784980 words/s, in_qsiz

'/Users/bingchen/Desktop/MSc_project/models/embeding'

In [10]:
#3. Transform text data into vector data
#model = models.Word2Vec.load(model_dir)
#len(model.wv['to'] )#vector for a word
#del model
#model.wv.index2word[0]
#len(model.wv['a'])

2018-06-30 15:16:00,269 : INFO : loading Word2Vec object from /Users/bingchen/Desktop/MSc_project/models/embeding
2018-06-30 15:16:00,832 : INFO : loading wv recursively from /Users/bingchen/Desktop/MSc_project/models/embeding.wv.* with mmap=None
2018-06-30 15:16:00,833 : INFO : loading vectors from /Users/bingchen/Desktop/MSc_project/models/embeding.wv.vectors.npy with mmap=None
2018-06-30 15:16:00,927 : INFO : setting ignored attribute vectors_norm to None
2018-06-30 15:16:00,928 : INFO : loading vocabulary recursively from /Users/bingchen/Desktop/MSc_project/models/embeding.vocabulary.* with mmap=None
2018-06-30 15:16:00,929 : INFO : loading trainables recursively from /Users/bingchen/Desktop/MSc_project/models/embeding.trainables.* with mmap=None
2018-06-30 15:16:00,930 : INFO : loading syn1neg from /Users/bingchen/Desktop/MSc_project/models/embeding.trainables.syn1neg.npy with mmap=None
2018-06-30 15:16:01,142 : INFO : setting ignored attribute cum_table to None
2018-06-30 15:16:0

In [48]:
#build the matrix for all vocab in training set(include 'pad' char and 'UNK') 
#matrix: (num_vocab+2) * embedding_size = (51917 + 2) * 100
#word_embedding.build_matrix(model_dir,vocab_tr_dir)
vocab_matrix = pd.read_csv(vocab_matrix_dir)

In [56]:
vocab_matrix.iloc[10]

Unnamed: 0         date
0             -0.488167
1               3.41892
2               3.57833
3              -4.67533
4              -2.56395
5             -0.992831
6               3.59923
7              -1.05731
8               1.29881
9             -0.120446
10             -3.42549
11             -1.30721
12              1.74788
13             -1.33895
14            -0.903042
15             -2.83713
16             -1.16866
17            0.0286798
18              2.30416
19             -1.95748
20             -1.56061
21            -0.988117
22             -2.84913
23             -4.53829
24             -4.15855
25              4.10495
26            -0.472795
27              5.10512
28              6.62414
                ...    
70              3.95966
71              3.55771
72             -8.28838
73              2.37476
74             -2.97004
75                2.832
76              2.96246
77              -3.2966
78              9.72396
79             0.845466
80             -

In [10]:
#4. num of labels
dficd_codes_filtered = pd.read_csv(code_filtered_dir,dtype={"ICD9_CODE": str})
icd9_codes = dficd_codes_filtered['ICD9_CODE'].unique() #len(icd9_codes):8908

In [3]:
#5. load all description of labels
des_dic = load_code_description.load_description()
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------

14567it [00:00, 61446.71it/s]
3882it [00:00, 62336.48it/s]
100%|██████████| 21163/21163 [00:00<00:00, 231855.58it/s]


In [3]:
des_dic = load_code_description.load_description()
#dficd_codes_filtered = pd.read_csv(code_filtered_dir,dtype={"ICD9_CODE": str})
#dficd_codes_filtered = dficd_codes_filtered.fillna('Miss')
#icd9_codes = list(dficd_codes_filtered['ICD9_CODE'].unique())
icd9_codes = load_code_description.load_all_codes()

14567it [00:00, 60224.63it/s]
3882it [00:00, 54667.54it/s]
100%|██████████| 21163/21163 [00:00<00:00, 146492.26it/s]


In [6]:
#len(icd9_codes) #num_labels

# Get sub dataset

In [21]:
counts = Counter()
dfnl = pd.read_csv('%s/ALL_CODES_NOTES.csv' % dev_data_dir)
for i in range(dfnl.shape[0]):
    for label in str(dfnl.iloc[i][3]).split(';'):
        counts[label] += 1
codes_50 = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
codes_50 = [code[0] for code in codes_50[:50]]

In [23]:
with open('%s/TOP_%s_CODES.csv' % (data_dir, str(50)), 'w') as of:
    w = csv.writer(of)
    for code in codes_50:
        w.writerow([code])

In [28]:
for splt in ['train', 'dev', 'test']:
    print(splt)
    hadm_ids = set()
    with open('%s/%s_50_hadm_ids.csv' % (data_dir, splt), 'r') as f:
        for line in f:
            hadm_ids.add(line.rstrip())
    with open('%s/notes_labeled.csv' % data_dir, 'r') as f:
        with open('%s/%s_%s.csv' % (data_dir, splt, str(50)), 'w') as of:
            r = csv.reader(f)
            w = csv.writer(of)
            #header
            w.writerow(next(r))
            i = 0
            for row in r:
                hadm_id = row[1]
                if hadm_id not in hadm_ids:
                    continue
                codes = set(str(row[3]).split(';'))
                filtered_codes = codes.intersection(set(codes_50))
                if len(filtered_codes) > 0:
                    w.writerow(row[:3] + [';'.join(filtered_codes)])
                    i += 1

train
dev
test


In [ ]:
for splt in ['train', 'dev', 'test']:
    filename = '%s/%s_%s.csv' % (data_dir, splt, str(50))
    df = pd.read_csv(filename)
    df['length'] = df.apply(lambda row: len(str(row['TEXT']).split()), axis=1)
    df = df.sort_values(['length'])
    df.to_csv('%s/%s_%s.csv' % (data_dir, splt, str(50)), index=False)